# Assignment: Data Wrangling

**Q1.** Open the "tidy_data.pdf" document in the repo, which is a paper called Tidy Data by Hadley Wickham.

  1. Read the abstract. What is this paper about?
    
    This paper is about the importance of data tidying in relation to data cleaning. The author also discusses that there is a little research on optimizing the effciency of data cleaning, even though it is a time-consuming aspect of data analysis. This paper is a case study that eliminates the repetitive tasks associated with data cleaning.
    
  2. Read the introduction. What is the "tidy data standard" intended to accomplish?

    The tidy data standard is intended to facilitate initial exploration and analysis of the data. The tidy data standard is designed to be a organization, structure that makes data easier to clean.

  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."

  The first sentence is all datasets are different, therefore you will face unique challenges or different experiences with any dataset you work with. The second setnece indicates that while it is often straightforward to identify the observations or variables, coming up with general definitions is more difficult because datasets make the context of the data vary.

  4. Read Section 2.2. How does Wickham define values, variables, and observations?

    Wickham defines values as typically either a number or strings. A variable has all values that measure the same underlying attribute across units, whereas an observation contains all vallues measured on the same unit across attributes.

  5. How is "Tidy Data" defined in section 2.3?

    Tiny data is definied as a standard way of mapping the meaning of a dataset to its structure. Tiny data consists of each variable forming a column, each observation forming a row, and each type of observational unit forms a table.

  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?

    The five most common problems with messy datasets are column headers being values, multiple variables being stored in one column, variables being stored in both rows and column, multiple types of obsesrvational units are stored in the same table, and a single observational unit being stored in multiple tables. The data in table 4 is messy because the columns are actually variables. Melting a dataset is when you turn columns into rows.

  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?

    Table 11 is messy because some of its columns at the top are values. Table 12 on the otherhand is tidy because none of the entries are variables.

  8. Read Section 6. What is the "chicken-and-egg" problem with focusing on tidy data? What does Wickham hope happens in the future with further work on the subject of data wrangling?

    The chicken and egg problem is that in order to be tidy, the tools that are used to work with the data also have to be too. Wickham hopes that in the future that tidy data can conceptually as a whole, be effective and that more tools and ideas will be created in the data science field.  

**Q2.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
2. Categorical variable: For the `./data/sharks.csv` data covered in the lecture, clean the "Type" variable as well as you can, and explain the choices you make.
3. Dummy variable: For the pretrial data covered in the lecture, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

In [20]:
import pandas as pd
import numpy as np
import seaborn as sns

Q2.1

In [ ]:
df = pd.read_csv('airbnb_hw.csv', low_memory=False)
print(df.shape, '\n')
df.head()

In [ ]:
price = df['Price']
price.unique()

In [ ]:
price = df['Price']
price = price.str.replace(',','') # To get rid of commas, which are unnecessary
price = pd.to_numeric(price,errors='coerce') # To change string to float because they are number values
print(price.unique() , '\n')
print('Total missing: ', sum( price.isnull() ) )

Q2.2

In [ ]:
df = pd.read_csv('sharks.csv', low_memory=False)
# df.head()
# df.columns.tolist()
df['Type'].value_counts()

In [ ]:
type = df['Type']

type = type.replace(['Sea Disaster', 'Boat','Boating','Boatomg'],'Watercraft') # Organizing very similar values into one column
type.value_counts()
type = type.replace(['Invalid', 'Questionable','Unconfirmed','Unverified','Under investigation'],np.nan) #  Organizing very similar values into one column
type.value_counts()

df['Type'] = type
del type
df['Type'].value_counts()

df['Fatal (Y/N)'] = df['Fatal (Y/N)'].replace(['UNKNOWN', 'F','M','2017'],np.nan) # Removing unexpected values
df['Fatal (Y/N)'] = df['Fatal (Y/N)'].replace('y','Y') # Capitalizing Y
pd.crosstab(df['Type'],df['Fatal (Y/N)'],normalize='index')

Q2.3

In [51]:
df = pd.read_parquet('justice_data.parquet')

In [ ]:
released = df['WhetherDefendantWasReleasedPretrial']
released = released.replace(9,np.nan) # 9 are unclear in codebook, so getting rid of
print(released.value_counts(),'\n')
sum(released.isnull()) # There are 31 missing values
df['WhetherDefendantWasReleasedPretrial'] = released # Replace data column with cleaned values
del released

Q2.4

In [ ]:
sentence = df['ImposedSentenceAllChargeInContactEvent']
type = df['SentenceTypeAllChargesAtConvictionInContactEvent']

sentence = pd.to_numeric(sentence,errors='coerce') # Changing values to numeric
sentence_NA = sentence.isnull()
print( np.sum(sentence_NA),'\n') # Lots of missing values

sentence = sentence.mask( type == 4, 0) # Replacing 4 with 0 because all cases are dismissed with 4
sentence = sentence.mask( type == 9, np.nan)

sentence_NA = sentence.isnull()
print( pd.crosstab(sentence_NA, type), '\n')
print( np.sum(sentence_NA),'\n')

df['ImposedSentenceAllChargeInContactEvent'] = sentence # Replace data with cleaned version
del sentence, type

**Q3.** Many important datasets contain a race variable, typically limited to a handful of values often including Black, White, Asian, Latino, and Indigenous. This question looks at data gathering efforts on this variable by the U.S. Federal government.

1. How did the most recent US Census gather data on race?
2. Why do we gather these data? What role do these kinds of data play in politics and society? Why does data quality matter?
3. Please provide a constructive criticism of how the Census was conducted: What was done well? What do you think was missing? How should future large scale surveys be adjusted to best reflect the diversity of the population? Could some of the Census' good practices be adopted more widely to gather richer and more useful data?
4. How did the Census gather data on sex and gender? Please provide a similar constructive criticism of their practices.
5. When it comes to cleaning data, what concerns do you have about protected characteristics like sex, gender, sexual identity, or race? What challenges can you imagine arising when there are missing values? What good or bad practices might people adopt, and why?
6. Suppose someone invented an algorithm to impute values for protected characteristics like race, gender, sex, or sexuality. What kinds of concerns would you have?